In [2]:
import pandas as pd


In [17]:
df_accepted=pd.read_csv("accepted_2007_to_2018Q4.csv",low_memory=True)
df_rejected=pd.read_csv("rejected_2007_to_2018Q4.csv",low_memory=True)

C:\Users\avida\AppData\Local\Temp\ipykernel_37396\1033986561.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accepted=pd.read_csv("accepted_2007_to_2018Q4.csv",low_memory=True)


In [3]:
df_rejected.head() 

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [4]:
df_accepted.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)

In [18]:

# === Step 1: Clean rejected data ===
# Drop unwanted columns
rejected = df_rejected.drop(columns=["State", "Zip Code"], errors="ignore")

# Add label
rejected["accepted_or_rejected"] = "rejected"

# === Step 2: Rename columns in accepted to match rejected ===
rename_map = {
    "loan_amnt": "Amount Requested",
    "issue_d": "Application Date",
    "title": "Loan Title",
    "dti": "Debt-To-Income Ratio",
    "emp_length": "Employment Length",
    "policy_code": "Policy Code",
    "fico_range_high": "Risk_Score"
}

accepted = df_accepted.rename(columns=rename_map)

# Drop unwanted columns
accepted = accepted.drop(columns=["addr_state", "zip_code"], errors="ignore")

# === Step 3: Add extra accepted-only columns ===
extra_cols = [
    "term", "int_rate", "installment", "grade", "sub_grade", "emp_title",
    "home_ownership", "annual_inc", "loan_status", "purpose", "delinq_2yrs",
    "out_prncp", "recoveries", "num_actv_bc_tl", "pub_rec_bankruptcies", "debt_settlement_flag"
]

# Keep only columns present in accepted
extra_cols = [col for col in extra_cols if col in accepted.columns]

# Combine renamed + extra columns
final_accepted_cols = list(rename_map.values()) + extra_cols

# Filter accepted data
accepted_filtered = accepted[final_accepted_cols].copy()
accepted_filtered["accepted_or_rejected"] = "accepted"

# === Step 4: Align columns between accepted and rejected ===
# Ensure rejected has the same columns (fill missing with NaN)
for col in accepted_filtered.columns:
    if col not in rejected.columns:
        rejected[col] = pd.NA

# Reorder rejected to match accepted
rejected = rejected[accepted_filtered.columns]

# === Step 5: Combine ===
df_combined = pd.concat([accepted_filtered, rejected], ignore_index=True)

# Save final dataset
df_combined.to_csv("cleaned_combined_lendingclub.csv", index=False)
print("✅ Saved to cleaned_combined_lendingclub.csv")


C:\Users\avida\AppData\Local\Temp\ipykernel_37396\497444407.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([accepted_filtered, rejected], ignore_index=True)


✅ Saved to cleaned_combined_lendingclub.csv


In [ ]:
df_combined.columns

In [3]:
df_accepted=pd.read_csv("accepted_2007_to_2018Q4.csv",low_memory=True)

C:\Users\avida\AppData\Local\Temp\ipykernel_34444\2351027926.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accepted=pd.read_csv("accepted_2007_to_2018Q4.csv",low_memory=True)


In [5]:
rename_map = {
    "loan_amnt": "Amount Requested",
    "issue_d": "Application Date",
    "title": "Loan Title",
    "dti": "Debt-To-Income Ratio",
    "emp_length": "Employment Length",

}

accepted = df_accepted.rename(columns=rename_map)

In [11]:
df_form=accepted[["Amount Requested", "Loan Title", "emp_title", "Employment Length",
    "home_ownership", "annual_inc", "purpose", "Debt-To-Income Ratio",
    "delinq_2yrs", "num_actv_bc_tl", "pub_rec_bankruptcies", "term","int_rate","loan_status"]]
df_form.head()

,Amount Requested,Loan Title,emp_title,Employment Length,home_ownership,annual_inc,purpose,Debt-To-Income Ratio,delinq_2yrs,num_actv_bc_tl,pub_rec_bankruptcies,term,int_rate,loan_status
0,3600.0,Debt consolidation,leadman,10+ years,MORTGAGE,55000.0,debt_consolidation,5.91,0.0,2.0,0.0,36 months,13.99,Fully Paid
1,24700.0,Business,Engineer,10+ years,MORTGAGE,65000.0,small_business,16.06,1.0,5.0,0.0,36 months,11.99,Fully Paid
2,20000.0,NaN,truck driver,10+ years,MORTGAGE,63000.0,home_improvement,10.78,0.0,2.0,0.0,60 months,10.78,Fully Paid
3,35000.0,Debt consolidation,Information Systems Officer,10+ years,MORTGAGE,110000.0,debt_consolidation,17.06,0.0,4.0,0.0,60 months,14.85,Current
4,10400.0,Major purchase,Contract Specialist,3 years,MORTGAGE,104433.0,major_purchase,25.37,1.0,4.0,0.0,60 months,22.45,Fully Paid


In [12]:
df_form.to_csv("form_data.csv",index=False)